In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx
import os
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset, download_url

from IPython.display import Javascript
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv,SAGEConv
from torch_geometric.nn import global_mean_pool

import wandb

In [2]:
test_patients = set(['F117', 'F130', 'F128', 'F133', 'F135', 'F138', 'F139', 'F136',
                 'F131', 'F344', 'F504', 'F510', 'M512', 'F341', 'F337', 'F324',
                 'F308', 'F317'])
elecs = ['Fp1', 'Fp2', 'Fpz', 'F3', 'F4', 'Fz', 'C3', 'C4',
         'Cz', 'P3', 'P4', 'Pz', '01', '02', '0z', 'F7', 
         'F8', 'T3', 'T4','T5', 'T6']

In [3]:
wandb.init(project = "neuroimaging_gnn_eeg_final_project", entity = "dmasny")

wandb: Currently logged in as: dmasny. Use `wandb login --relogin` to force relogin


In [4]:
class GNNDataset(InMemoryDataset):
    
    def __init__(self, 
                 root, 
                 data_dict, 
                 idx, 
                 feature_names, 
                 allow_loops, 
                 weighted, 
                 threshold = 0.65,
                 transform = None, 
                 pre_transform = None, 
                 pre_filter = None):
        
        self.data = np.load(data_dict, allow_pickle = True).item()
        self.stage = idx # idx 0 - train, idx 1 - test
        self.feature_names = feature_names
        self.allow_loops = allow_loops
        self.weighted = weighted
        self.threshold = threshold
        
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[self.stage])
        
    def vectorize_adj_mat_coo(self, matrix, allow_loops = True):
        source_nodes = []
        target_nodes = []
        if allow_loops:
            for i in range(matrix.shape[0]):
                for j in range(i, matrix.shape[1]):
                    source_nodes.append(i)
                    target_nodes.append(j)
        else:
            for i in range(matrix.shape[0]):
                for j in range(i + 1, matrix.shape[1]):
                    source_nodes.append(i)
                    target_nodes.append(j)
        return source_nodes, target_nodes

    def vectorize_adj_mat_weights(self, matrix):
        edge_weights = []
        if self.weighted == 'weighted':
            if self.allow_loops:
                for i in range(matrix.shape[0]):
                    for j in range(i, matrix.shape[1]):
                        edge_weights.append(matrix[i][j])
            else:
                for i in range(matrix.shape[0]):
                    for j in range(i + 1, matrix.shape[1]):
                        edge_weights.append(matrix[i][j])
        else:
            threshold = np.min(np.max(matrix, axis = 1)) if self.weighted == 'unweighted_dynamic_threshold' else self.threshold
            mask = np.array((matrix > threshold), dtype = np.uint8)
            if allow_loops:
                for i in range(mask.shape[0]):
                    for j in range(i, mask.shape[1]):
                        edge_weights.append(mask[i][j])
            else:
                for i in range(mask.shape[0]):
                    for j in range(i + 1, mask.shape[1]):
                        edge_weights.append(mask[i][j])
        return edge_weights

    def upload_data(self, patch_name):
        '''
        input:
            path_to_data: path to precomputed node representations
            path_to_adj_matr: path to precomputed adj matrices
            
        returns:
            Pygeometric Data object (see PyG docs)
        '''
        X, target, adj_matrix = self.data[patch_name] # triplet in format [X, target, A]
        edge_index = np.array(self.vectorize_adj_mat_coo(adj_matrix))
        edge_features = self.vectorize_adj_mat_weights(adj_matrix)
        return Data(x = torch.tensor(X), 
                    edge_index = torch.tensor(edge_index),
                    edge_attrs = edge_features, 
                    y = torch.tensor([target]))  
    
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return [f'gnn_dataset_train_{self.feature_names}.pt',
                f'gnn_dataset_test_{self.feature_names}.pt']

    def download(self):
        pass
        
    def process(self):
        data_list = []
        for elem in self.data.keys():
            data_list.append(self.upload_data(elem))
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[self.stage])
    

In [5]:
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, output_dim = 2):
        super(GCN, self).__init__()
        torch.manual_seed(2022)
        self.conv1 = SAGEConv(num_node_features, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.linear = Linear(hidden_channels, output_dim)

    def forward(self, x, edge_index, batch):
        
        x = self.conv1(x, edge_index)
        x = x.relu()
        
        x = self.conv2(x, edge_index)
        x = x.relu()
        
        x = self.conv3(x, edge_index)
        x = x.relu()
        
        x = self.conv4(x ,edge_index)
        x = x.relu()
        

        # Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        # Classifier
        x = self.linear(x)
        
        return x


In [6]:
# train_dataset = GNNDataset('./', 'gnn_prepared_data/power_and_entropy_gnn_train.npy', 
#                            0, 
#                            'all_features',
#                            allow_loops = True,
#                            weighted = 'weighted') # idx = 0 - train
# test_dataset = GNNDataset('./', 'gnn_prepared_data/power_and_entropy_gnn_test.npy', 
#                           1, 
#                           'all_features',
#                           allow_loops = True,
#                           weighted = 'weighted') # idx = 1 - train


In [7]:
# train_dataset[0]

In [8]:
# train_dataset = GNNDataset('./', 'gnn_prepared_data/power_and_entropy_gnn_train.npy', 0, 'all_features',  weighted = False) # idx = 0 - train
# test_dataset = GNNDataset('./', 'gnn_prepared_data/power_and_entropy_gnn_test.npy', 1, 'all_features', weighted = False) # idx = 1 - train


1) adj(2) + loops(2) + weighted(2) + data(3) = 2*2*2*3 = 12 exp
2) batch size, num_layers, num_epoches, lr, scheduler
3)

object : [X, target, A]
X - only use local info about the node
1) acc entropy <
2) acc power <
3) acc power + entropy < gnn(power + entropy + adj matrix)

In [9]:
# def train(model, dataloader):
#     model.train()

#     for data in dataloader:  # Iterate in batches over the training dataset.
#         out = model(data.x.type(dtype=torch.float), data.edge_index, data.batch)  # Perform a single forward pass.
#         loss = criterion(out, data.y)  # Compute the loss.
#         loss.backward()  # Derive gradients.
#         optimizer.step()  # Update parameters based on gradients.
#         optimizer.zero_grad()  # Clear gradients.
#         wandb.log()
        
# def test(model, dataloader):
#     model.eval()
#     correct = 0
#     for data in dataloader:  # Iterate in batches over the training/test dataset.
#         out = model(data.x.type(dtype=torch.float), data.edge_index, data.batch)  
#         pred = out.argmax(dim=1)  # Use the class with highest probability.
#         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
#     return correct / len(loader.dataset)  # Derive ratio of correct predictions.

In [10]:
def train(model, 
          epoches, 
          criterion, 
          train_dataloader, 
          test_dataloader, 
          expirement_name, 
          path_to_save_weights = 'model_weights'):
    
    best_accuracy = 0
    for i in range(epoches):
        model.train()
        train_correct = 0
        train_loss = 0
        for data in train_dataloader:
            out = model(data.x.type(dtype = torch.float), 
                        data.edge_index, 
                        data.batch)  
            loss = criterion(out, data.y) 
            train_loss += loss
            pred = out.argmax(dim = 1)
            train_correct += int((pred == data.y).sum())
            loss.backward() 
            optimizer.step()  
            optimizer.zero_grad() 
        train_loss = train_loss / len(train_dataloader.dataset)
        train_accuracy = train_correct / len(train_dataloader.dataset)
        
        wandb.log({'train_loss': train_loss})
        wandb.log({'train_accuracy': train_accuracy})
        
        model.eval()
        with torch.no_grad():  
            test_correct = 0
            for data in test_dataloader:  
                out = model(data.x.type(dtype = torch.float), data.edge_index, data.batch)  
                pred = out.argmax(dim = 1)  
                test_correct += int((pred == data.y).sum())
            test_accuracy = test_correct / len(test_dataloader.dataset)
            
            if test_accuracy > best_accuracy:
                best_accuracy = test_accuracy
                torch.save(model.state_dict(), f'{path_to_save_weights}/{expirement_name}.pth')
                
            print(f'Epoch:{i} Train acc:{train_accuracy} Test acc:{test_accuracy} Train loss:{train_loss}')
            
            wandb.log({'test_accuracy': test_accuracy})
            wandb.log({'best_test_accuracy': best_accuracy})
            
    
    wandb.finish()

In [ ]:
loops = [True, False]
weighted = ['weighted', 'unweighted_dynamic_threshold', 'unweighted_static_threshold']
data = ['power_and_entropy', 'only_entropy', 'only_powerbands']

lr = 0.001
wd = 0.001
batch_size = 50
hidden_channels = 128 
epoches = 1000


# train_dataset = GNNDataset('./', 'gnn_prepared_data/power_and_entropy_gnn_train.npy', 
#                            0, 
#                            'all_features',
#                            allow_loops = True,
#                            weighted = 'weighted')
# test_dataset = GNNDataset('./', 'gnn_prepared_data/power_and_entropy_gnn_test.npy', 
#                           1, 
#                           'all_features',
#                           allow_loops = True,
#                           weighted = 'weighted') 

# model = GCN(num_node_features = train_dataset.num_node_features, 
#                             hidden_channels = hidden_channels)
# optimizer = torch.optim.AdamW(model.parameters(), lr = lr, weight_decay = wd)
# criterion = torch.nn.CrossEntropyLoss()


# train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
# test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)
    
# train(model = model,
#       epoches = epoches, 
#       criterion = criterion, 
#       train_dataloader = train_loader, 
#       test_dataloader = test_loader,
#       expirement_name = 'debug')

buff = []
for loops_config in loops:
    for weight_config in weighted:
        for dataset in data:
            train_dataset = GNNDataset(root = './', 
                                       data_dict = f'gnn_prepared_data/{dataset}_gnn_train.npy',
                                       idx = 0, 
                                       feature_names = dataset, 
                                       allow_loops = loops_config, 
                                       weighted = weight_config) # idx = 0 - train
            test_dataset = GNNDataset(root = './', 
                                      data_dict = f'gnn_prepared_data/{dataset}_gnn_test.npy', 
                                      idx = 1, 
                                      feature_names = dataset,
                                      allow_loops = loops_config, 
                                      weighted = weight_config) # idx = 1 - train

            train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
            test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

            model = GCN(num_node_features = train_dataset.num_node_features, 
                        hidden_channels = hidden_channels)
            optimizer = torch.optim.AdamW(model.parameters(), lr = lr, weight_decay = wd)
            criterion = torch.nn.CrossEntropyLoss()


            config = {
                        'learning_rate': lr,
                        'weight_decay': wd,
                        'epochs': epoches,
                        'training_batch_size' : batch_size,
                        'validation_batch_size' : batch_size,
                        'loops_config': loops_config,
                        'weight_config': weight_config,
                        'dataset': dataset,
                        'criterion': criterion,    
                        'node_representation_size': train_dataset.num_node_features, 
                        'model': {
                                    'num_graph_conv_blocks': 4,
                                    'hidden_channels' : hidden_channels,
                                    'activation' : 'ReLU',
                                    'readout': 'global_mean_pool'}}
#             buff.append(config)
#             print(config)
            experiment_name = f'self_loops={loops_config}_weighted={weight_config}_data={dataset}'

            wandb.init(project = 'neuroimaging_gnn_eeg_final_project', 
                       entity = 'dmasny',
                       name = experiment_name, 
                       config = config)

            train(model = model,
                  epoches = epoches, 
                  criterion = criterion, 
                  train_dataloader = train_loader, 
                  test_dataloader = test_loader,
                  expirement_name = experiment_name)
            print(experiment_name)
            del model


Processing...
Done!
Processing...
Done!


Epoch:0 Train acc:0.49415204678362573 Test acc:0.4988479262672811 Train loss:0.014838864095509052
Epoch:1 Train acc:0.5112085769980507 Test acc:0.5011520737327189 Train loss:0.014280869625508785
Epoch:2 Train acc:0.5082846003898636 Test acc:0.4988479262672811 Train loss:0.014163868501782417
Epoch:3 Train acc:0.49171539961013644 Test acc:0.4988479262672811 Train loss:0.014291990548372269
Epoch:4 Train acc:0.5160818713450293 Test acc:0.5011520737327189 Train loss:0.01417999155819416
Epoch:5 Train acc:0.5384990253411306 Test acc:0.5011520737327189 Train loss:0.014096962288022041
Epoch:6 Train acc:0.5014619883040936 Test acc:0.5011520737327189 Train loss:0.014203084632754326
Epoch:7 Train acc:0.5307017543859649 Test acc:0.5011520737327189 Train loss:0.014203093945980072
Epoch:8 Train acc:0.5389863547758285 Test acc:0.5023041474654378 Train loss:0.014072886668145657
Epoch:9 Train acc:0.5477582846003899 Test acc:0.5702764976958525 Train loss:0.013991250656545162
Epoch:10 Train acc:0.54775828

Epoch:85 Train acc:0.6159844054580896 Test acc:0.6059907834101382 Train loss:0.01263479981571436
Epoch:86 Train acc:0.6179337231968811 Test acc:0.6140552995391705 Train loss:0.012479064054787159
Epoch:87 Train acc:0.6208576998050682 Test acc:0.6082949308755761 Train loss:0.01242883875966072
Epoch:88 Train acc:0.6033138401559455 Test acc:0.6105990783410138 Train loss:0.012819865718483925
Epoch:89 Train acc:0.6067251461988304 Test acc:0.6071428571428571 Train loss:0.012421331368386745
Epoch:90 Train acc:0.6154970760233918 Test acc:0.6036866359447005 Train loss:0.012563303112983704
Epoch:91 Train acc:0.6169590643274854 Test acc:0.6278801843317973 Train loss:0.012347953394055367
Epoch:92 Train acc:0.6072124756335283 Test acc:0.5898617511520737 Train loss:0.01241233292967081
Epoch:93 Train acc:0.6062378167641326 Test acc:0.6129032258064516 Train loss:0.013049634173512459
Epoch:94 Train acc:0.618421052631579 Test acc:0.6117511520737328 Train loss:0.012483673170208931
Epoch:95 Train acc:0.624

Epoch:169 Train acc:0.6033138401559455 Test acc:0.6267281105990783 Train loss:0.01231867540627718
Epoch:170 Train acc:0.6169590643274854 Test acc:0.5944700460829493 Train loss:0.012439655140042305
Epoch:171 Train acc:0.6276803118908382 Test acc:0.5748847926267281 Train loss:0.012395798228681087
Epoch:172 Train acc:0.6228070175438597 Test acc:0.631336405529954 Train loss:0.01267055980861187
Epoch:173 Train acc:0.6242690058479532 Test acc:0.6140552995391705 Train loss:0.012327016331255436
Epoch:174 Train acc:0.6154970760233918 Test acc:0.6370967741935484 Train loss:0.012280918657779694
Epoch:175 Train acc:0.6164717348927875 Test acc:0.6382488479262672 Train loss:0.012414082884788513
Epoch:176 Train acc:0.6213450292397661 Test acc:0.6152073732718893 Train loss:0.012228062376379967
Epoch:177 Train acc:0.6081871345029239 Test acc:0.6405529953917051 Train loss:0.012546336278319359
Epoch:178 Train acc:0.6237816764132553 Test acc:0.6175115207373272 Train loss:0.012655978091061115
Epoch:179 Tra

Epoch:253 Train acc:0.6369395711500975 Test acc:0.6082949308755761 Train loss:0.011919207870960236
Epoch:254 Train acc:0.6423001949317739 Test acc:0.6036866359447005 Train loss:0.01202597189694643
Epoch:255 Train acc:0.6403508771929824 Test acc:0.6486175115207373 Train loss:0.011973198503255844
Epoch:256 Train acc:0.648635477582846 Test acc:0.6405529953917051 Train loss:0.011811149306595325
Epoch:257 Train acc:0.6447368421052632 Test acc:0.5944700460829493 Train loss:0.011656787246465683
Epoch:258 Train acc:0.6471734892787524 Test acc:0.6267281105990783 Train loss:0.011858430691063404
Epoch:259 Train acc:0.5326510721247564 Test acc:0.586405529953917 Train loss:0.014502390287816525
Epoch:260 Train acc:0.5779727095516569 Test acc:0.543778801843318 Train loss:0.01378484908491373
Epoch:261 Train acc:0.6232943469785575 Test acc:0.6428571428571429 Train loss:0.012760789133608341
Epoch:262 Train acc:0.6301169590643275 Test acc:0.5518433179723502 Train loss:0.012575024738907814
Epoch:263 Train

...

Epoch:168 Train acc:0.8089668615984406 Test acc:0.4216589861751152 Train loss:0.008005909621715546
Epoch:169 Train acc:0.8118908382066277 Test acc:0.47235023041474655 Train loss:0.007823743857443333
Epoch:170 Train acc:0.814327485380117 Test acc:0.4804147465437788 Train loss:0.008076589554548264
Epoch:171 Train acc:0.804093567251462 Test acc:0.4815668202764977 Train loss:0.00836227647960186
Epoch:172 Train acc:0.8133528265107213 Test acc:0.48502304147465436 Train loss:0.008077058009803295
Epoch:173 Train acc:0.7846003898635477 Test acc:0.4642857142857143 Train loss:0.009167632088065147
Epoch:174 Train acc:0.7860623781676414 Test acc:0.47465437788018433 Train loss:0.008931182324886322
Epoch:175 Train acc:0.8128654970760234 Test acc:0.45622119815668205 Train loss:0.007892485707998276
Epoch:176 Train acc:0.8240740740740741 Test acc:0.45622119815668205 Train loss:0.007877664640545845
Epoch:177 Train acc:0.8089668615984406 Test acc:0.44930875576036866 Train loss:0.00826746691018343
Epoch:17

Epoch:251 Train acc:0.8966861598440545 Test acc:0.4700460829493088 Train loss:0.004721750505268574
Epoch:252 Train acc:0.922514619883041 Test acc:0.47465437788018433 Train loss:0.004250993020832539
Epoch:253 Train acc:0.9064327485380117 Test acc:0.48502304147465436 Train loss:0.004585830494761467
Epoch:254 Train acc:0.8942495126705653 Test acc:0.45622119815668205 Train loss:0.005072996485978365
Epoch:255 Train acc:0.9078947368421053 Test acc:0.4596774193548387 Train loss:0.004597974009811878
Epoch:256 Train acc:0.9117933723196882 Test acc:0.48847926267281105 Train loss:0.004388578236103058
Epoch:257 Train acc:0.9137426900584795 Test acc:0.4504608294930876 Train loss:0.004338076803833246
Epoch:258 Train acc:0.9093567251461988 Test acc:0.4735023041474654 Train loss:0.005096870474517345
Epoch:259 Train acc:0.8762183235867447 Test acc:0.45852534562211983 Train loss:0.006569830700755119
Epoch:260 Train acc:0.8771929824561403 Test acc:0.48847926267281105 Train loss:0.005446721334010363
Epoch

Epoch:334 Train acc:0.9615009746588694 Test acc:0.4769585253456221 Train loss:0.0019236438674852252
Epoch:335 Train acc:0.9712475633528265 Test acc:0.4815668202764977 Train loss:0.0019237229134887457
Epoch:336 Train acc:0.9751461988304093 Test acc:0.4608294930875576 Train loss:0.001654534018598497
Epoch:337 Train acc:0.9702729044834308 Test acc:0.4642857142857143 Train loss:0.0017411510925740004
Epoch:338 Train acc:0.9780701754385965 Test acc:0.48847926267281105 Train loss:0.001587913022376597
Epoch:339 Train acc:0.982943469785575 Test acc:0.4815668202764977 Train loss:0.0013652225024998188
Epoch:340 Train acc:0.9751461988304093 Test acc:0.45622119815668205 Train loss:0.0015946993371471763
Epoch:341 Train acc:0.9702729044834308 Test acc:0.478110599078341 Train loss:0.0016276683891192079
Epoch:342 Train acc:0.9824561403508771 Test acc:0.4861751152073733 Train loss:0.0013597295619547367
Epoch:343 Train acc:0.9766081871345029 Test acc:0.4769585253456221 Train loss:0.0015141029143705964
Ep

Epoch:417 Train acc:0.8961988304093568 Test acc:0.4665898617511521 Train loss:0.006633682642132044
Epoch:418 Train acc:0.9405458089668616 Test acc:0.49078341013824883 Train loss:0.0028197967913001776
Epoch:419 Train acc:0.9785575048732943 Test acc:0.4873271889400922 Train loss:0.0015456335386261344
Epoch:420 Train acc:0.9853801169590644 Test acc:0.5092165898617511 Train loss:0.001115648658014834
Epoch:421 Train acc:0.9897660818713451 Test acc:0.49193548387096775 Train loss:0.0011312095448374748
Epoch:422 Train acc:0.9775828460038987 Test acc:0.4942396313364055 Train loss:0.0015220020432025194
Epoch:423 Train acc:0.9907407407407407 Test acc:0.48963133640552997 Train loss:0.000937771750614047
Epoch:424 Train acc:0.9727095516569201 Test acc:0.4804147465437788 Train loss:0.0019488069228827953
Epoch:425 Train acc:0.9848927875243665 Test acc:0.47235023041474655 Train loss:0.0010453700087964535
Epoch:426 Train acc:0.99317738791423 Test acc:0.4827188940092166 Train loss:0.00071114202728495
Epo

Epoch:504 Train acc:0.9990253411306043 Test acc:0.48502304147465436 Train loss:0.00016449153190478683
Epoch:505 Train acc:1.0 Test acc:0.47465437788018433 Train loss:0.00011814207391580567
Epoch:506 Train acc:1.0 Test acc:0.4873271889400922 Train loss:9.756950748851523e-05
Epoch:507 Train acc:0.9990253411306043 Test acc:0.478110599078341 Train loss:0.0001360599126201123
Epoch:508 Train acc:0.9995126705653021 Test acc:0.47580645161290325 Train loss:0.00017112036584876478
Epoch:509 Train acc:0.9936647173489279 Test acc:0.4423963133640553 Train loss:0.0004843349743168801
Epoch:510 Train acc:0.9736842105263158 Test acc:0.4435483870967742 Train loss:0.001493989140726626
Epoch:511 Train acc:0.9429824561403509 Test acc:0.4976958525345622 Train loss:0.003587373998016119
Epoch:512 Train acc:0.9259259259259259 Test acc:0.5149769585253456 Train loss:0.004227769561111927
Epoch:513 Train acc:0.9273879142300195 Test acc:0.49539170506912444 Train loss:0.004162126686424017
Epoch:514 Train acc:0.968323

In [19]:
!tree processed/

processed/
├── gnn_dataset_test_all_features.pt
├── gnn_dataset_train_all_features.pt
├── pre_filter.pt
└── pre_transform.pt

0 directories, 4 files


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
